In [ ]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import LdaMulticore
import gensim
import spacy 
import nltk 
from nltk import ngrams
import re
#from utilities import * 

In [ ]:

#load the cleaned and preprocessed dataset 
ArNews_df_update = pd.read_csv("/Users/AlaAlBinSaleh/Desktop/Desktop/Capstone/ArNews_df_Cleaned.csv", index_col=[0])

In [9]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

#word embedding

In [8]:

#1)Word Embding using AraVec 
model = gensim.models.Word2Vec.load("full_grams_cbow_100_wiki/full_grams_cbow_100_wiki.mdl")

In [10]:
#test the model vocbulary 
print("We've",len(model.wv.index_to_key),"vocabularies")

We've 662109 vocabularies


In [ ]:
%mkdir spacyModel

mkdir: spacyModel: File exists


In [ ]:
model.wv.save_word2vec_format("./spacyModel/aravec.txt")

In [ ]:
!gzip ./spacyModel/aravec.txt

In [ ]:
#!python3 -m spacy init ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz
!python3 -m spacy init vectors ar ./spacyModel/aravec.txt.gz ./spacy.aravec.model
#spacy.aravec.model --vectors-loc ./

ℹ Creating blank nlp object for language 'ar'
[2021-11-12 23:16:03,453] [INFO] Reading vectors from spacyModel/aravec.txt.gz
662109it [01:21, 8129.57it/s]
[2021-11-12 23:17:24,926] [INFO] Loaded vectors from spacyModel/aravec.txt.gz
✔ Successfully converted 662109 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/Users/AlaAlBinSaleh/Desktop/Desktop/Capstone/spacy.aravec.model


In [11]:
nlp = spacy.load("./spacy.aravec.model")

In [12]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [14]:
nlp.tokenizer = Preprocessor(nlp.tokenizer)
nlp.tokenizer

In [15]:
#Test the word embedding 
nlp("بيانات").vector

array([ 1.3541306 , -0.47691634, -2.037478  ,  1.7818414 ,  3.2348447 ,
        2.9923422 , -1.6732128 ,  2.8066268 , -0.13688694,  1.7843444 ,
        0.9327977 , -0.9078639 , -1.3284395 , -0.6065882 ,  3.0176415 ,
       -4.276657  ,  1.2542839 ,  0.89644   ,  4.40036   , -3.3811958 ,
        3.0062063 , -1.9969931 ,  2.0136075 ,  0.8734018 ,  5.452243  ,
        1.7340939 ,  1.3925358 , -1.7027333 , -0.56593776, -3.1578138 ,
       -2.6459064 , -1.2861159 ,  2.8269248 , -2.9818544 ,  3.3686252 ,
       -2.1550925 ,  0.562423  , -1.5678904 ,  2.0604756 ,  1.0104055 ,
        0.949106  ,  3.5613863 ,  0.21240458,  0.69437015,  0.9959716 ,
       -1.5333009 ,  1.1857115 , -0.6239423 ,  0.79202735, -3.4440598 ,
        2.4681935 , -0.19734618, -1.057888  ,  2.6490996 , -3.813241  ,
       -6.690389  , -1.9947684 , -0.01682599,  3.6795888 , -1.0036598 ,
       -0.08111735, -4.285988  ,  0.13309926, -3.1279614 , -4.398629  ,
        2.2460012 ,  6.380341  , -1.0859216 ,  1.4137076 ,  1.11

In [16]:
model.wv.most_similar("بيانات")

[('البيانات', 0.8408971428871155),
 ('معلومات', 0.7972483038902283),
 ('قاعده_بيانات', 0.7843179702758789),
 ('حسابات', 0.7556501626968384),
 ('المعلومات', 0.7542430758476257),
 ('قاعده_البيانات', 0.7380597591400146),
 ('للبيانات', 0.7246543169021606),
 ('رسائل_البريد_الالكتروني', 0.7245200276374817),
 ('قواعد_بيانات', 0.7238749861717224),
 ('وبيانات', 0.7214061617851257)]

In [ ]:
model.wv["بيانات"]

array([ 1.3541306 , -0.47691634, -2.037478  ,  1.7818414 ,  3.2348447 ,
        2.9923422 , -1.6732128 ,  2.8066268 , -0.13688694,  1.7843444 ,
        0.9327977 , -0.9078639 , -1.3284395 , -0.6065882 ,  3.0176415 ,
       -4.276657  ,  1.2542839 ,  0.89644   ,  4.40036   , -3.3811958 ,
        3.0062063 , -1.9969931 ,  2.0136075 ,  0.8734018 ,  5.452243  ,
        1.7340939 ,  1.3925358 , -1.7027333 , -0.56593776, -3.1578138 ,
       -2.6459064 , -1.2861159 ,  2.8269248 , -2.9818544 ,  3.3686252 ,
       -2.1550925 ,  0.562423  , -1.5678904 ,  2.0604756 ,  1.0104055 ,
        0.949106  ,  3.5613863 ,  0.21240458,  0.69437015,  0.9959716 ,
       -1.5333009 ,  1.1857115 , -0.6239423 ,  0.79202735, -3.4440598 ,
        2.4681935 , -0.19734618, -1.057888  ,  2.6490996 , -3.813241  ,
       -6.690389  , -1.9947684 , -0.01682599,  3.6795888 , -1.0036598 ,
       -0.08111735, -4.285988  ,  0.13309926, -3.1279614 , -4.398629  ,
        2.2460012 ,  6.380341  , -1.0859216 ,  1.4137076 ,  1.11

In [32]:
content = ArNews_df_update["Detokenize"].values

In [38]:
def Word_embedding(text):
    global word_embeddings
    word_embeddings = []
    for line in ArNews_df_update["content"]:
        avgword2vec = None
        counter = 0
        #split into sentences 
        for word in line.split():
            if word in model.wv.key_to_index:
                counter += 1
                # words embedding of sentences 
                if avgword2vec is None:
                    avgword2vec = model.wv[word]
                else:
                    avgword2vec = avgword2vec + model.wv[word]
        if avgword2vec is not None:
            avgword2vec = avgword2vec / counter # normalize sum
            word_embeddings.append(avgword2vec)
    return word_embeddings

In [39]:
vec =  Word_embedding(content)

In [76]:
#vec

In [87]:
#model.wv.key_to_index

-----------------------------------------------------------------------

In [ ]:
output = re.sub(r'\s*[A-Za-z]+\b', '' , text)


In [64]:
corpus=[]
word=[]

for i in range(len(ArNews_df_update['Detokenize'])):
        word =ArNews_df_update['Detokenize'].iloc[i]
        corpus.append(word)

In [86]:
texts = [[word for word in str(document).lower().split()]for document in corpus]

In [70]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
#print(dictionary)


Dictionary(45199 unique tokens: ['أضف', 'ابن', 'اخر', 'اذ', 'اضة']...)


In [85]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [83]:
lda = gensim.models.LdaModel(corpus, id2word=dictionary, num_topics=10)

In [84]:
#print(lda.print_topics(num_topics=10, num_words=15), "\n")
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.018*"دول" + 0.012*"امر" + 0.011*"اير" + 0.010*"تفق" + 0.010*"تحد" + 0.010*"حكم" + 0.009*"عمل" + 0.009*"سيس" + 0.008*"رئس" + 0.008*"قرر"
Topic: 1 
Words: 0.023*"شرك" + 0.016*"نسب" + 0.015*"سوق" + 0.014*"سعر" + 0.013*"رفع" + 0.012*"سهم" + 0.012*"عمل" + 0.011*"ريل" + 0.010*"قطع" + 0.010*"ائة"
Topic: 2 
Words: 0.026*"جمع" + 0.020*"وزر" + 0.017*"عمل" + 0.015*"طلب" + 0.014*"سعد" + 0.010*"درس" + 0.010*"قدم" + 0.009*"خدم" + 0.008*"خلل" + 0.008*"جلس"
Topic: 3 
Words: 0.014*"قتل" + 0.012*"نظم" + 0.010*"يمن" + 0.010*"عمل" + 0.010*"سور" + 0.010*"حوث" + 0.009*"قوت" + 0.009*"نطق" + 0.009*"عسكر" + 0.008*"دعش"
Topic: 4 
Words: 0.033*"سعد" + 0.022*"امر" + 0.021*"حمد" + 0.019*"رئس" + 0.018*"ملك" + 0.017*"سلم" + 0.016*"عبدالعزيز" + 0.015*"شرف" + 0.014*"الل" + 0.013*"سمو"
Topic: 5 
Words: 0.019*"عمل" + 0.018*"نطق" + 0.011*"طرق" + 0.010*"خدم" + 0.009*"حفظ" + 0.008*"بلغ" + 0.008*"نفذ" + 0.007*"خلل" + 0.007*"جمع" + 0.007*"دير"
Topic: 6 
Words: 0.027*"لعب" + 0.026*"فرق" + 0.018*"ندي" + 0.01